<a href="https://colab.research.google.com/github/SHAILESHYADA/Stocks-Data-Manipulation/blob/main/Stock_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy

# Data loading and cleaning
df = pd.read_csv('/content/stocks.csv')

In [2]:
df.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [5]:
import pandas as pd
import os

# Task 1: Data Exploration and Manipulation

# Part a: Read the data into a Pandas DataFrame
stocks_df = pd.read_csv('stocks.csv')

# Part b: Calculate the average closing price for each unique ticker
average_close = stocks_df.groupby('Name')['close'].mean().reset_index()
average_close.columns = ['Name', 'avgPrice']
average_close.to_csv('stock_data.csv', index=False)

# Part c: Calculate the daily percentage change in the closing price for each ticker and save separately
tickers = stocks_df['Name'].unique()
if not os.path.exists('data'):
    os.makedirs('data')

for ticker in tickers:
    ticker_df = stocks_df[stocks_df['Name'] == ticker].copy()
    ticker_df['pct_change'] = ticker_df['close'].pct_change()
    ticker_df.to_csv(f'data/{ticker}.csv', index=False)

# Task 2: Time Series Analysis

# Part a: Resample the data to a monthly frequency and save separately
if not os.path.exists('monthData'):
    os.makedirs('monthData')

for ticker in tickers:
    ticker_df = stocks_df[stocks_df['Name'] == ticker].copy()
    ticker_df['date'] = pd.to_datetime(ticker_df['date'])
    ticker_df.set_index('date', inplace=True)
    monthly_resampled = ticker_df.resample('M').agg({
        'open': 'mean',
        'high': 'mean',
        'low': 'mean',
        'close': 'mean',
        'volume': 'sum'
    }).reset_index()
    monthly_resampled.to_csv(f'monthData/{ticker}.csv', index=False)

# Part b: Calculate the rolling 30-day average of the closing price for each ticker and add a column
for ticker in tickers:
    ticker_df = pd.read_csv(f'data/{ticker}.csv')
    ticker_df['rolling_avg'] = ticker_df['close'].rolling(window=30).mean()
    ticker_df.to_csv(f'data/{ticker}.csv', index=False)

# Task 3: Data Merging and Joining

# Part a: Read both files into separate Pandas DataFrames
fundamentals_df = pd.read_csv('fundamentals.csv')
prices_df = pd.read_csv('prices.csv')

# Part b: Merge the two DataFrames on the Ticker column
merged_df = pd.merge(fundamentals_df, prices_df, on='Ticker')

# Part c: Calculate the market capitalization to price ratio for each ticker and save
merged_df['MarketCap_to_Price'] = merged_df['Marketcap'] / merged_df['Currentprice']
merged_df.to_csv('stockInfo.csv', index=False)

# Task 4: Performance Analysis

# Part a: Read the data of each ticker into a Pandas DataFrame and calculate daily profit/loss
for ticker in tickers:
    ticker_df = pd.read_csv(f'data/{ticker}.csv')
    ticker_df['pnl'] = ticker_df['close'] - ticker_df['close'].iloc[0]

    # Part c: Calculate and add cumulative profit/loss column
    ticker_df['cumulative_pnl'] = ticker_df['pnl'].cumsum()

    # Part d: Calculate and add the drawdown column
    ticker_df['drawdown'] = ticker_df['pnl'] - ticker_df['pnl'].cummax()

    # Save the updated DataFrame
    ticker_df.to_csv(f'data/{ticker}.csv', index=False)

